# GermlineMasterWorkflow.wdl output check development notes
## (Not a cheesy indictment of any known programmer or programming organization)

In [ ]:
import os
import sys
import pandas as pd
import numpy as np
x_dir = '/Users/yo/zzIForge/fullyJan10/'
if not os.path.isdir(x_dir):
    print('directory not found\n', x_dir)

****
## GermlineMasterWorkflow.wdl call sequence:
    * trimseq
    * align
    * merge
    * dedup
    * DAB
    * realign
    * bsqr
    * haplotype
    * vsqr
    * DHVC
****
****
### 1.0) TestTasks/Runtrim_sequences.wdl, workflow as CUTADAPTTRIM.RunTrimSequencesTask (as trimseq)
    * passes NormalInputReads as InputReads
### 1.5 ) "trimseq"  calls Tasks/trim_sequences.wdl workflow as TRIMSEQ.trimsequencesTask as either:
#### 1.5.1 ) TRIMSEQ_paired
#### 1.5.2 ) TRIMSEQ_single
##### >> by either name they (it) runs:
    * src/shell/shell_preamble.sh as "BashPreamble"
    * src/shell/trim_sequences.sh as "TrimSeqScript"
        * calling src/shell/shared_functions.sh as "BashSharedFunctions"
##### >> with parameters:
    * PairedEnd (Boolean)
    * InputRead1 (File)
    * InputRead2 (String)
    * SampleName (String)
    * Adapters (File)
    * CutAdapt (String) == path
    * CutAdaptThreads (String) == integer
    * TrimEnvProfile (File)
    * DebugMode (String) == DNE
##### >> and orphan parameters:
    * CutAdaptThreads aka cpu
    * TrimSoftMemLimit aka s_vmem
    * TrimHardMemLimit aka h_vmem
### 1.5.8 ) Named output
    * Outputs Array[File]
    * using (unix, linux, ? dakine) "glob" function 
### 1.5.9 ) Output assumption is optional input for "align"

****
### 2.0 ) TestTasks/Runalignment.wdl, workflow as ALIGNMENT.RunAlignmentTask (as align)
### 2.5 ) "align" calls Tasks/alignment.wdl, workflow as ALIGN.alignmentTask (as ALIGN_paired)
#### 2.5.1 ) ALIGN_paired runs:
    * src/shell/shell_preamble.sh as "BashPreamble"
    * src/shell/alignment.sh as "AlignmentScript"
        * /usr/local/apps/bioapps/sentieon/sentieon-genomics-201808.01 as "Sentieon"
        * src/shell/shared_functions.sh as "BashSharedFunctions"
##### >> with parameters:
    * PairedEnd (Boolean)
    * InputRead1 (File)
    * InputRead2 (String)
    * SampleName (String)
    * Platform (String)
    * Library (String)
    * PlatformUnit (String)
    * CenterName (String)
    * Sentieon (String) == path
    * SentieonThreads (String) == integer
    * AlignEnvProfile (File)
    * ChunkSizeInBases (String) 
    * BWAExtraOptionsString (String)
    
##### >> and orphan parameters:
    * SentieonThreads aka cpu
    * AlignSoftMemLimit aka s_vmem
    * AlignHardMemLimit aka h_vmem

### 2.5.7 ) Named output inner:
    * OutputBams: SampleName.aligned.sorted.bam
    * OutputBais: SampleName.aligned.sorted.bam.bai
### 2.5.8 ) Named output middle:
    * OutputBams: Array[File]
    * OutputBais: Array[File]
### 2.5.9 ) Output assumption is inputs for "merge"

4) GermlineMasterWorkflow.wdl calls Tasks/merge_aligned_bam.wdl runs:
    * MergeBamScript: src/shell/merge_bams.sh (using shared_functions.sh)
    * with Config/MergeBamEnvProfile.file
    * Warns: cpu, s_vmem, h_vmem
    * Puts Out:
        * OutputBams: SampleName.aligned.sorted.merged.bam
        * OutputBais: SampleName.aligned.sorted.merged.bam.bai
        
5) GermlineMasterWorkflow.wdl calls Tasks/dedup.wdl runs:
    * DedupScript: src/shell/dedup.sh (using shared_functions.sh)
    * with Config/DedupEnvProfile.file
    * Warns: cpu, s_vmem, h_vmem
    * Puts Out:
        * OutputBams: SampleName.aligned.sorted.deduped.bam
        * OutputBais: SampleName.aligned.sorted.deduped.bam.bai
        
6) GermlineMasterWorkflow.wdl calls Tasks/deliver_alignment.wdl runs:
    * DeliveryAlignment_Script: src/shell/deliver_alignment.sh (using shared_functions.sh)
    * using: GermlineMasterWorkflow.FilledIn.json
    * no output but: dowop Delivery/Alignment
    
7) GermlineMasterWorkflow.wdl calls Tasks/realignment.wdl runs:
    * RealignmentScript: src/shell/realignment.sh (using shared_functions.sh)
    * using: Reference/Mills_and_1000G_gold_standard.indels.hg38.vcf
    * RealignEnvProfile: Config/RealignEnvProfile.file
    * Warns: cpu, s_vmem, h_vmem
    * Puts Out:
        * OutputBams: SampleName.aligned.sorted.deduped.realigned.bam
        * OutputBais: SampleName.aligned.sorted.deduped.realigned.bam.bai
        
8) GermlineMasterWorkflow.wdl calls Tasks/bqsr.wdl runs:
    * BqsrScript: src/shell/bqsr.sh (
    * using: 
        * Reference/Mills_and_1000G_gold_standard.indels.hg38.vcf
        * Reference/dbsnp_138.hg38.vcf
        * Config/BqsrEnvProfile.file
    * Warns: cpu, s_vmem, h_vmem
    * Puts Out:
        * RecalTable: SampleName.recal_data.table
        
9) GermlineMasterWorkflow.wdl calls Tasks/haplotyper.wdl runs:
    * HaplotyperScript: src/shell/haplotyper.sh (using shared_functions.sh)
    * using:
        * HaplotyperExtraOptionsString: (magic string)
        * Config/HaplotyperEnvProfile.file
        * Config/HaplotyperVCFSourceField.file
    * Warns: cpu, s_vmem, h_vmem
    * Puts Out:
        * OutputVcf: SampleName.vcf
        * OutputVcfIdx: SampleName.vcf.idx"
        
10) GermlineMasterWorkflow.wdl calls Tasks/vqsr.wdl runs:
    * VqsrScript: src/shell/vqsr.sh 
    * using magic strings:
        * VqsrSnpResourceString
        * VqsrIndelResourceString
        * AnnotateText
    * using:
        * InputVcf && InputVcfIdx (input output from) VCF from Haplotyper 
        * Config/VqsrEnvProfile.file
    * Warns: cpu, s_vmem, h_vmem
    * Puts Out:
        * OutputVcf: SampleName.SNP.recaled.vcf
        * OutputVcfIdx: SampleName.SNP.recaled.vcf.idx
        
10) GermlineMasterWorkflow.wdl calls Tasks/deliver_HaplotyperVC.wdl runs:
    * DeliveryHaplotyperVC_Script: src/shell/deliver_haplotyperVC.sh (using shared_functions.sh)
    * to dakine: Delivery/HaplotyperVC
    * using: GermlineMasterWorkflow.FilledIn.json    

In [ ]:
import os
import sys
import numpy as np
import pandas as pd

import knpackage.toolbox as kn

In [5]:
json_dir = '../data/Jsons'
json_farameters_file = 'GermlineMasterWorkflow.FilledIn.json'

json_parameters = kn.get_run_parameters(json_dir, json_farameters_file)
dakine = 'WorkflowJson'
lineno = 0
for k, v in json_parameters.items():
    if dakine in k:
        print('%i %30s\n\t%s\n'%(lineno, k, v))
    lineno += 1

7 GermlineMasterWF.DAB.WorkflowJson
	/projects/bioinformatics/DEL/Jsons/GermlineMasterWorkflow.FilledIn.json

14 GermlineMasterWF.DHVC.WorkflowJson
	/projects/bioinformatics/DEL/Jsons/GermlineMasterWorkflow.FilledIn.json

